# **Chapter 14: Heaps and Priority Queues**

> *"Priority is not just about what you do first—it's about what you do most efficiently."* — Anonymous

---

## **14.1 Introduction to Priority Queues**

A **priority queue** is an abstract data type where each element has a priority, and the element with the highest (or lowest) priority is always removed first. Unlike a FIFO queue, the order of removal is based on priority, not insertion order.

Priority queues are fundamental in many algorithms:

- **Dijkstra's shortest path** (extract node with minimum distance)
- **Prim's minimum spanning tree** (extract edge with minimum weight)
- **Huffman coding** (extract two smallest frequencies)
- **Event-driven simulation** (process next event in time order)
- **Load balancing** (serve highest priority task)
- **Scheduling** (CPU, disk I/O)

### **14.1.1 Heap: The Classic Implementation**

A **heap** is the most common data structure for implementing a priority queue. It is a complete binary tree with the **heap property**:

- **Max-heap**: Parent key ≥ children keys (largest at root)
- **Min-heap**: Parent key ≤ children keys (smallest at root)

```
Min-heap:       2
              /   \
             5     8
            / \   /
           9  10 12

Max-heap:      12
              /  \
             10   8
            / \   /
           5  9  2
```

---

## **14.2 Binary Heaps**

Binary heaps are the simplest and most widely used heap variant. They use an array to store the complete binary tree level by level.

### **14.2.1 Array Representation**

For a node at index `i` (0-based):
- Parent: `(i-1)//2`
- Left child: `2*i + 1`
- Right child: `2*i + 2`

The array representation is space-efficient and provides good cache locality.

```
Heap as tree:       2
                   / \
                  5   8
                 / \   \
                9  10  12

Array:  [2, 5, 8, 9, 10, 12]
Index:   0  1  2  3   4   5
```

### **14.2.2 Binary Heap Implementation (Min-Heap)**

```python
from typing import List, Optional, Callable

class MinHeap:
    def __init__(self):
        self.heap = []

    def parent(self, i: int) -> int:
        return (i - 1) // 2

    def left_child(self, i: int) -> int:
        return 2 * i + 1

    def right_child(self, i: int) -> int:
        return 2 * i + 2

    def swap(self, i: int, j: int) -> None:
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def peek(self) -> Optional[int]:
        """Return minimum element without removing."""
        return self.heap[0] if self.heap else None

    def size(self) -> int:
        return len(self.heap)
```

#### **Insert (push)**

Insert at the end, then **sift up** (or bubble up) to restore heap property.

```python
def insert(self, key):
    self.heap.append(key)
    self._sift_up(len(self.heap) - 1)

def _sift_up(self, i):
    """Move element at i up until heap property restored."""
    while i > 0 and self.heap[self.parent(i)] > self.heap[i]:
        self.swap(i, self.parent(i))
        i = self.parent(i)
```

**Time complexity:** O(log n) worst-case, due to tree height.

#### **Extract Minimum (pop)**

Remove root, replace with last element, then **sift down** (or bubble down).

```python
def extract_min(self) -> Optional[int]:
    if not self.heap:
        return None
    if len(self.heap) == 1:
        return self.heap.pop()

    root = self.heap[0]
    self.heap[0] = self.heap.pop()  # move last to root
    self._sift_down(0)
    return root

def _sift_down(self, i):
    n = len(self.heap)
    while True:
        smallest = i
        left = self.left_child(i)
        right = self.right_child(i)

        if left < n and self.heap[left] < self.heap[smallest]:
            smallest = left
        if right < n and self.heap[right] < self.heap[smallest]:
            smallest = right

        if smallest != i:
            self.swap(i, smallest)
            i = smallest
        else:
            break
```

**Time complexity:** O(log n)

#### **Heapify: Building a Heap from an Array**

We can build a heap in O(n) by applying sift-down from the last non-leaf node down to root.

```python
def build_heap(self, arr: List[int]) -> None:
    """Build heap from arbitrary list in O(n)."""
    self.heap = arr[:]
    n = len(self.heap)
    # Start from last non-leaf node
    for i in range(n // 2 - 1, -1, -1):
        self._sift_down(i)
```

**Why O(n)?** The number of nodes at height h is at most n/2^(h+1), and sift-down at height h takes O(h). Summation leads to O(n).

### **14.2.3 Max-Heap**

Max-heap implementation is symmetric: reverse comparison operators.

```python
class MaxHeap:
    # ... same structure, but with > comparisons in sift_up and sift_down
```

### **14.2.4 Heap Sort**

Heap sort uses a heap to sort an array in-place:

1. Build max-heap from array (O(n))
2. Repeatedly swap root (maximum) with last element, reduce heap size, and sift down new root.

```python
def heap_sort(arr: List[int]) -> None:
    n = len(arr)

    # Build max heap
    for i in range(n // 2 - 1, -1, -1):
        _sift_down(arr, n, i)

    # Extract elements one by one
    for i in range(n - 1, 0, -1):
        arr[0], arr[i] = arr[i], arr[0]  # swap
        _sift_down(arr, i, 0)

def _sift_down(arr, n, i):
    """Sift down for max-heap on array segment of size n."""
    largest = i
    left = 2*i + 1
    right = 2*i + 2
    if left < n and arr[left] > arr[largest]:
        largest = left
    if right < n and arr[right] > arr[largest]:
        largest = right
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        _sift_down(arr, n, largest)
```

**Time complexity:** O(n log n) – always (no worst-case degradation).  
**Space complexity:** O(1) (in-place).

---

## **14.3 Priority Queue Operations with Heap**

| Operation   | Binary Heap Time |
|-------------|------------------|
| Insert      | O(log n)         |
| Extract Min | O(log n)         |
| Peek        | O(1)             |
| Build       | O(n)             |
| Decrease Key| O(log n) (if we can locate node) |

**Decrease key** requires knowing the node's position. In binary heap without a map, we'd need to search (O(n)). Usually, we maintain a separate dictionary mapping values to indices for efficient updates.

---

## **14.4 d-ary Heaps**

A **d-ary heap** generalizes binary heap: each node has up to d children. For d > 2, the tree is shallower (height = log_d n), which can reduce the number of comparisons for sift-down. However, sift-up may require more comparisons (d per level).

- **Insert:** O(log_d n)
- **Extract-min:** O(d log_d n)

Choosing d trades off between these operations. d is often chosen to match cache line size (e.g., 4 or 8).

Implementation changes: children indices for node i are `d*i + 1` through `d*i + d`.

---

## **14.5 Binomial Heaps**

A **binomial heap** consists of a collection of binomial trees, each satisfying the min-heap property. Binomial trees are defined recursively:

- A binomial tree of order 0 is a single node.
- A binomial tree of order k has a root whose children are binomial trees of orders 0, 1, ..., k-1.

```
B0: ●

B1: ●
     |
     ●

B2:  ●
    / \
   ●   ●
   |
   ●

B3:  ●
   / | \
  ●  ●  ●
  |  |
  ●  ●
  |
  ●
```

**Properties:**
- A binomial tree of order k has exactly 2^k nodes.
- Height = k.
- Root has k children.
- Merging two trees of same order yields one tree of order k+1.

### **14.5.1 Binomial Heap Structure**

A binomial heap is a forest of binomial trees, each obeying min-heap property, and no two trees have the same order. It is stored as a linked list of roots sorted by increasing order.

```python
class BinomialNode:
    def __init__(self, key):
        self.key = key
        self.degree = 0          # number of children
        self.parent = None
        self.child = None        # pointer to first child
        self.sibling = None      # right sibling in root list or sibling list

class BinomialHeap:
    def __init__(self):
        self.head = None         # head of root list
```

### **14.5.2 Operations**

#### **Merge (Union)**

Union of two binomial heaps is analogous to binary addition. We merge root lists by increasing order, combining trees of same degree.

```python
def merge(self, other):
    """Merge other heap into this one. Other becomes empty."""
    # First, merge root lists sorted by degree
    self.head = self._merge_roots(self.head, other.head)
    other.head = None
    if not self.head:
        return

    # Traverse and combine trees of same degree
    prev = None
    curr = self.head
    next_node = curr.sibling

    while next_node:
        if (curr.degree != next_node.degree or
            (next_node.sibling and next_node.sibling.degree == curr.degree)):
            # Move forward if degrees differ or three consecutive same degree
            prev = curr
            curr = next_node
        else:
            # Combine curr and next_node (curr becomes root)
            if curr.key <= next_node.key:
                curr.sibling = next_node.sibling
                self._link(next_node, curr)
            else:
                if prev:
                    prev.sibling = next_node
                else:
                    self.head = next_node
                self._link(curr, next_node)
                curr = next_node
        next_node = curr.sibling

def _link(self, child, root):
    """Make child a child of root (root assumed smaller key)."""
    child.parent = root
    child.sibling = root.child
    root.child = child
    root.degree += 1
```

#### **Insert**

Insert a node by creating a new heap and merging.

```python
def insert(self, key):
    new_heap = BinomialHeap()
    new_heap.head = BinomialNode(key)
    self.merge(new_heap)
```

#### **Extract Minimum**

Find the tree with minimum root, remove it, and merge its children (which form a valid binomial heap) back.

```python
def extract_min(self):
    if not self.head:
        return None

    # Find minimum root
    min_prev = None
    min_node = self.head
    prev = None
    curr = self.head
    while curr:
        if curr.key < min_node.key:
            min_node = curr
            min_prev = prev
        prev = curr
        curr = curr.sibling

    # Remove min_node from root list
    if min_prev:
        min_prev.sibling = min_node.sibling
    else:
        self.head = min_node.sibling

    # Reverse order of min_node's children (they form binomial heap of increasing degree)
    child = min_node.child
    child_heap = BinomialHeap()
    while child:
        next_child = child.sibling
        child.sibling = child_heap.head
        child.parent = None
        child_heap.head = child
        child = next_child

    # Merge child heap with current heap
    self.merge(child_heap)
    return min_node.key
```

**Complexities (amortized):**
- Insert: O(1)
- Merge: O(log n)
- Extract-min: O(log n)

---

## **14.6 Fibonacci Heaps**

Fibonacci heaps are even more sophisticated, providing **amortized O(1) insert, merge, and decrease-key**, and **O(log n) extract-min**. They are used in Dijkstra's and Prim's algorithms where many decrease-key operations occur.

### **14.6.1 Structure**

- A collection of heap-ordered trees (roots in a circular doubly linked list).
- Each node has pointers: parent, child (one child), left/right siblings, degree, and a boolean `marked` (for cascading cuts).
- The root list is a circular doubly linked list.
- The heap maintains a pointer `min` to the minimum root.

### **14.6.2 Key Operations**

#### **Insert**

Create a new node, add to root list, update min.

#### **Merge (Union)**

Concatenate root lists, update min.

#### **Extract Minimum**

Remove min node, add its children to root list, then **consolidate** trees of same degree by linking roots with larger keys under smaller ones (like binomial heap union). Consolidation uses an array of size O(log n).

#### **Decrease Key**

Decrease the key of a node. If heap property violated (node smaller than parent), cut the node and add it to root list. If the parent was already marked, cut parent as well (cascading cut). Mark nodes when they lose a child.

#### **Delete**

Decrease key to -∞, then extract min.

### **14.6.3 Amortized Analysis**

Using potential function Φ = number of trees + 2 * number of marked nodes, we can prove:

- Insert: O(1) actual, ΔΦ = +1 → amortized O(1)
- Merge: O(1) actual, Φ unchanged → O(1)
- Extract-min: O(trees + degrees) actual, ΔΦ reduces → amortized O(log n)
- Decrease-key: O(c) actual (c cascading cuts), ΔΦ decreases → amortized O(1)

### **14.6.4 Why Not Always Use Fibonacci Heaps?**

- High constant factors
- Complex implementation
- Not cache-friendly
- In practice, binary heaps often outperform for moderate n

---

## **14.7 Pairing Heaps**

Pairing heaps are a simpler self-adjusting heap with good amortized performance. They are represented as trees where each node has a `leftmost child` and `right sibling`.

Operations:

- **Insert:** Make node a new tree, meld with heap.
- **Merge (meld):** Compare roots, link smaller as root of larger.
- **Extract-min:** Remove root, then merge its subtrees in pairs (pairing) and then merge results.

Analysis: All operations O(log n) amortized, but decrease-key is conjectured O(1) but not proven.

```python
class PairingNode:
    def __init__(self, key):
        self.key = key
        self.child = None
        self.sibling = None

class PairingHeap:
    def __init__(self):
        self.root = None

    def merge(self, a, b):
        if a is None: return b
        if b is None: return a
        if a.key < b.key:
            b.sibling = a.child
            a.child = b
            return a
        else:
            a.sibling = b.child
            b.child = a
            return b

    def insert(self, key):
        node = PairingNode(key)
        self.root = self.merge(self.root, node)

    def extract_min(self):
        if not self.root:
            return None
        min_val = self.root.key
        self.root = self._merge_pairs(self.root.child)
        return min_val

    def _merge_pairs(self, node):
        if not node or not node.sibling:
            return node
        a = node
        b = node.sibling
        rest = b.sibling
        a.sibling = None
        b.sibling = None
        return self.merge(self.merge(a, b), self._merge_pairs(rest))
```

---

## **14.8 Applications of Heaps**

### **14.8.1 Median Maintenance**

Maintain the median of a stream of numbers using two heaps: a max-heap for the lower half, and a min-heap for the upper half.

```python
import heapq

class MedianFinder:
    def __init__(self):
        self.small = []  # max-heap (store negatives)
        self.large = []  # min-heap

    def add_num(self, num):
        if len(self.small) == len(self.large):
            # Push to large, then move smallest from large to small
            heapq.heappush(self.large, num)
            heapq.heappush(self.small, -heapq.heappop(self.large))
        else:
            heapq.heappush(self.small, -num)
            heapq.heappush(self.large, -heapq.heappop(self.small))

    def find_median(self):
        if len(self.small) > len(self.large):
            return -self.small[0]
        else:
            return (-self.small[0] + self.large[0]) / 2.0
```

### **14.8.2 Top-K Problems**

Find the k largest elements in a stream using a min-heap of size k.

```python
def top_k(nums, k):
    heap = []
    for num in nums:
        if len(heap) < k:
            heapq.heappush(heap, num)
        elif num > heap[0]:
            heapq.heapreplace(heap, num)
    return heap
```

### **14.8.3 Dijkstra's Algorithm**

Priority queue extracts the node with smallest tentative distance. Using Fibonacci heap can improve theoretical complexity from O((V+E) log V) to O(V log V + E).

### **14.8.4 Huffman Coding**

Repeatedly extract two smallest frequencies, combine, and insert back.

---

## **14.9 Comparison of Heap Variants**

| Heap Type          | Insert   | Extract Min | Decrease Key | Merge     | Memory/Impl |
|--------------------|----------|-------------|--------------|-----------|-------------|
| Binary Heap        | O(log n) | O(log n)    | O(log n)*    | O(n)      | Simple, array |
| d-ary Heap         | O(log_d n)| O(d log_d n) | O(log_d n)* | O(n)      | Simple, array |
| Binomial Heap      | O(1) amort| O(log n)    | O(log n)     | O(log n)  | Moderate, pointers |
| Fibonacci Heap     | O(1) amort| O(log n) amort | O(1) amort | O(1) amort | Complex, pointers |
| Pairing Heap       | O(1) amort| O(log n) amort | O(log n)** | O(1) amort | Moderate, pointers |

\* Requires ability to locate node.  
\** Conjectured O(1) amortized.

---

## **14.10 Summary**

Heaps provide efficient priority queue operations. The binary heap is the workhorse for most applications due to its simplicity and good performance. For specialized needs, binomial and Fibonacci heaps offer better asymptotic bounds, at the cost of complexity.

```
─────────────────────────────────────────────────────────────────────
Operation               Binary Heap   Binomial   Fibonacci   Pairing
─────────────────────────────────────────────────────────────────────
Insert                  O(log n)      O(1)       O(1)        O(1)
Extract Min             O(log n)      O(log n)   O(log n)    O(log n)
Decrease Key            O(log n)*     O(log n)   O(1)        O(log n)**
Merge                   O(n)          O(log n)   O(1)        O(1)
─────────────────────────────────────────────────────────────────────
```

---

## **14.11 Practice Problems**

### **Problem 1: Kth Largest Element in a Stream**
Design a class that supports adding numbers and returning the kth largest element.

**Hint:** Use a min-heap of size k.

### **Problem 2: Merge k Sorted Lists**
Given k sorted linked lists, merge them into one sorted list.

**Hint:** Use a min-heap of (node value, list index, node).

### **Problem 3: Sliding Window Maximum**
Given an array and a window size k, find the maximum in each sliding window.

**Hint:** Use a max-heap with lazy deletion or a deque.

### **Problem 4: Find Median from Data Stream**
Implement as described in Section 14.8.1.

### **Problem 5: Task Scheduler**
Given tasks and a cooldown period, schedule tasks to minimize total time.

**Hint:** Use max-heap for most frequent tasks, and a queue for cooldown.

### **Problem 6: IPO**
Given k projects, each with capital and profit, and starting capital W, find maximum capital after at most k projects.

**Hint:** Use min-heap for projects affordable, max-heap for profits.

### **Problem 7: Reorganize String**
Rearrange characters of a string so that no two adjacent characters are the same.

**Hint:** Use max-heap by frequency.

---

## **14.12 Further Reading**

1. **"Introduction to Algorithms" (CLRS)** – Chapters 6 (Heapsort), 19 (Binomial Heaps), 20 (Fibonacci Heaps)
2. **"The Art of Computer Programming, Vol 3"** – Section 5.2.3 (Heapsort)
3. **"Algorithms"** by Robert Sedgewick – Chapter 2.4 (Priority Queues)
4. **"Data Structures and Algorithm Analysis in C++"** – Chapter 6 (Priority Queues)
5. **Original Papers**:
   - Vuillemin (1978) – Binomial heaps
   - Fredman, Tarjan (1987) – Fibonacci heaps
   - Fredman, Sedgewick, Sleator, Tarjan (1986) – Pairing heaps

---

> **Coming in Chapter 15**: **Graph Fundamentals** – We'll explore graph representations, types, and basic properties that form the foundation for graph algorithms.

---

**End of Chapter 14**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='13. specialized_trees.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. graph_theory_algorithms/15. graph_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
